---
title: "분포간 차이 구현 노트북"
date: "2025-08-12"
category: "Data Science"
tags: ["KDE", "Distribution Distance", "KL", "JS", "Wasserstein"]
excerpt: "분포간 차이 구현 노트북"
---

기본적인 설정들
- notebook kernel : docker jupyter lab kernel 사용

실험내용
> 분포 간에 차이를 측정할 떄 사용하는 지표 구현
- Kullback-Leibler Divergence
- Jensen-Shannon Divergence
- Wasserstein Distance

1. KDE를 통해 분포 추정
2. (사이즈별) 분포 간 차이 측정

# Estimate Density Function

In [6]:
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import StandardScaler
import numpy as np


# 자동 대역폭 선택을 위한 함수
def auto_bandwidth(data):
    """
    Silverman의 규칙을 사용하여 자동 대역폭 선택
    """
    n = len(data)
    std = np.std(data)

    # Silverman's rule of thumb
    bandwidth = 1.06 * std * (n ** (-1 / 5))
    return bandwidth


def kde_density_estimation(kde_data, bandwidth=20, kernel="gaussian"):
    """
    KDE를 사용하여 밀도 추정
    """
    kde = KernelDensity(bandwidth=bandwidth, kernel=kernel)
    kde.fit(kde_data)

    density_values = np.exp(kde.score_samples(kde_data))

    return kde, density_values


scaler = StandardScaler()

kde_data = data.drop(["size"], axis=1)
normalized_data = scaler.fit_transform(kde_data)


auto_bw = auto_bandwidth(normalized_data)
print(auto_bw)
kde, density_values = kde_density_estimation(normalized_data, bandwidth=auto_bw, kernel="gaussian")
print(density_values.shape)

0.20609596822753715
(3599,)


In [11]:
type(normalized_data)

numpy.ndarray

추정한 밀도 분포 통계값 뽑아서 확인해보기

In [7]:
# 밀도 값들의 기본 통계 확인
print("=== 밀도 값 현황 ===")
print(f"밀도 값 개수: {len(density_values)}")
print(f"최소값: {np.min(density_values):.6f}")
print(f"최대값: {np.max(density_values):.6f}")
print(f"평균값: {np.mean(density_values):.6f}")
print(f"중간값: {np.median(density_values):.6f}")
print(f"표준편차: {np.std(density_values):.6f}")
print()

# 0.1보다 큰 값이 있는지 확인
above_threshold = density_values[density_values > 0.1]
print(f"0.1보다 큰 값 개수: {len(above_threshold)}")
print(f"0.1보다 큰 값들: {above_threshold}")

=== 밀도 값 현황 ===
밀도 값 개수: 3599
최소값: 0.003901
최대값: 0.054179
평균값: 0.015022
중간값: 0.011848
표준편차: 0.010400

0.1보다 큰 값 개수: 0
0.1보다 큰 값들: []


특정 포인트에 대한 밀도 값을 해석하는 방법 중 하나

In [8]:
def interpret_density_value(density_values, target_density):
    """
    특정 밀도 값의 의미 해석
    """
    print(f"=== 밀도 값 {target_density} 해석 ===")

    # 1. 상대적 위치
    percentile = np.mean(density_values <= target_density) * 100
    print(f"상대적 위치: 하위 {percentile:.1f}%")

    # 2. 일반성 정도
    if percentile < 25:
        generality = "매우 특이함"
    elif percentile < 50:
        generality = "특이함"
    elif percentile < 75:
        generality = "일반적"
    else:
        generality = "매우 일반적"

    print(f"일반성 정도: {generality}")

    # 3. 유사한 밀도를 가진 데이터 수
    similar_count = np.sum(np.abs(density_values - target_density) < 0.005)
    print(f"유사한 밀도 데이터: {similar_count}개")

    return percentile, generality


# 해석 실행
percentile, generality = interpret_density_value(density_values, 0.02)

=== 밀도 값 0.02 해석 ===
상대적 위치: 하위 73.3%
일반성 정도: 일반적
유사한 밀도 데이터: 787개


# 사이즈 별 분포 차이 확인하기

In [13]:
# 1. 사이즈별 밀도 추정

import numpy as np
from scipy.stats import entropy
from sklearn.preprocessing import StandardScaler

from src.utils.dist_diff import auto_bandwidth, kde_density_estimation

scaler = StandardScaler()
data_12_1 = data.loc[data["size"] == "12.1"].drop(["size"], axis=1)
data_12_1 = scaler.fit_transform(data_12_1)

scaler = StandardScaler()
data_12_6 = data.loc[data["size"] == "12.6"].drop(["size"], axis=1)
data_12_6 = scaler.fit_transform(data_12_6)

scaler = StandardScaler()
data_13_2 = data.loc[data["size"] == "13.2"].drop(["size"], axis=1)
data_13_2 = scaler.fit_transform(data_13_2)


kde_12_1, density_values_12_1 = kde_density_estimation(data_12_1, bandwidth=auto_bandwidth(data_12_1), kernel="gaussian")
kde_12_6, density_values_12_6 = kde_density_estimation(data_12_6, bandwidth=auto_bandwidth(data_12_6), kernel="gaussian")
kde_13_2, density_values_13_2 = kde_density_estimation(data_13_2, bandwidth=auto_bandwidth(data_13_2), kernel="gaussian")

In [16]:
# 2. KL Divergence 계산

np.random.seed(42)

epsilon = 1e-10
p_hist = np.clip(density_values_12_1, epsilon, None)
q_hist = np.clip(density_values_12_6, epsilon, None)
q_hist = np.random.choice(q_hist, size=p_hist.shape[0])  # 사이즈 맞춰주기 위해서

kl_value = entropy(p_hist, q_hist)  # scipy의 entropy가 KL divergence 계산
print("KL Divergence (12.1 vs 12.6):", kl_value)
kl_value = entropy(q_hist, p_hist)  # scipy의 entropy가 KL divergence 계산
print("KL Divergence (12.6 vs 12.1):", kl_value)

KL Divergence (12.1 vs 12.6): 0.3568783475138717
KL Divergence (12.6 vs 12.1): 0.3461563588894394


KL Divergence 특징. 
- 순서 바꾸면 값이 달라짐
- noise에 취약함 혹은 민감함

In [18]:
# 3. JS Divergence 계산

import numpy as np
from scipy.stats import entropy


def js_divergence(p: np.ndarray, q: np.ndarray) -> float:
    """
    Jensen-Shannon Divergence 계산

    Args:
        p, q: 두 확률 분포 (1차원 배열)

    Returns:
        float: JS Divergence 값
    """
    # 배열 길이 맞추기
    min_length = min(len(p), len(q))

    p = np.random.choice(p, min_length)
    q = np.random.choice(q, min_length)

    # 확률 분포로 정규화 (합이 1이 되도록)
    p_norm = p / np.sum(p)
    q_norm = q / np.sum(q)

    # 중간 분포 m = (p + q) / 2
    m = (p_norm + q_norm) / 2

    # JS Divergence = (KL(p||m) + KL(q||m)) / 2
    js_div = (entropy(p_norm, m) + entropy(q_norm, m)) / 2

    return js_div


# 사용 예시
js_div_value = js_divergence(p_hist, q_hist)
print(f"JS Divergence: {js_div_value:.6f}")


JS Divergence: 0.084235


In [21]:
# 4. JS distance 계산


def js_distance(p: np.ndarray, q: np.ndarray) -> float:
    """
    Jensen-Shannon Distance 계산

    Args:
        p, q: 두 확률 분포 (1차원 배열)

    Returns:
        float: JS Distance 값 (0~1 범위)
    """
    # JS Divergence 계산
    js_div = js_divergence(p, q)

    # JS Distance = sqrt(JS Divergence)
    js_dist = np.sqrt(js_div)

    return js_dist


# 사용 예시
js_dist_value = js_distance(p_hist, q_hist)
print(f"JS Distance: {js_dist_value:.6f}")

# JS Distance는 0~1 범위
print("=== JS Distance 해석 가이드 ===")

if js_dist_value < 0.1:
    print("0.0 ~ 0.1: 매우 유사한 분포 (거의 동일)")
elif js_dist_value < 0.2:
    print("0.1 ~ 0.2: 유사한 분포 (비슷함)")
elif js_dist_value < 0.3:
    print("0.2 ~ 0.3: 약간 다른 분포 (차이가 있음)")
elif js_dist_value < 0.5:
    print("0.3 ~ 0.5: 상당히 다른 분포 (명확한 차이)")
elif js_dist_value < 0.7:
    print("0.5 ~ 0.7: 매우 다른 분포 (큰 차이)")
else:
    print("0.7 ~ 1.0: 완전히 다른 분포 (극도로 다름)")

JS Distance: 0.289960
=== JS Distance 해석 가이드 ===
0.2 ~ 0.3: 약간 다른 분포 (차이가 있음)


In [23]:
from scipy.stats import wasserstein_distance


def wasserstein_distance_scipy(p: np.ndarray, q: np.ndarray) -> float:
    """
    scipy를 사용한 Wasserstein Distance 계산

    수식: W(p,q) = Σ|x_i - y_i| / n
    """
    return wasserstein_distance(p, q)


wasserstein_distance(p_hist, q_hist)

np.float64(0.0005600815559936411)